# Book Recommendation Dataset: EDA & preparation 

Sources:
- dataset paper: [Ziegler, Cai-Nicolas, et al. "Improving recommendation lists through topic diversification."](https://dl.acm.org/doi/pdf/10.1145/1060745.1060754)
- [dataset kaggle](https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset/data)

In [ ]:
import os

import matplotlib.pyplot as plt
import missingno as mn
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib_venn import venn2
from pandas import DataFrame

In [ ]:
og_ds_path = "./og-dataset"
new_ds_path = "./book-feedback.csv"

In [ ]:
books_df = pd.read_csv(filepath_or_buffer=os.path.join(og_ds_path, "Books.csv"))
ratings_df = pd.read_csv(filepath_or_buffer=os.path.join(og_ds_path, "Ratings.csv"))
users_df = pd.read_csv(filepath_or_buffer=os.path.join(og_ds_path, "Users.csv"))

## Basic info

In [ ]:
books_df.describe()

In [ ]:
books_df.info()

In [ ]:
ratings_df.describe()

In [ ]:
ratings_df.info()

In [ ]:
ratings_df.head()

In [ ]:
users_df.describe()

In [ ]:
users_df.info()

## Missing values & errors

In [ ]:
def missing(df: DataFrame):
    print(df.isnull().sum())
    print(df[df.isnull().any(axis=1)])

for df in [books_df, ratings_df, users_df]:
    missing(df)
    print("="*40)

In [ ]:
# bacause of time constraints image data will not be used
# only few null values, no need for something complex
books_df.drop(["Image-URL-S", "Image-URL-M", "Image-URL-L"], axis=1, inplace=True)
books_df["Book-Author"].fillna("", inplace=True)
books_df["Publisher"].fillna("", inplace=True)


In [ ]:
joined_df = pd.merge(books_df, ratings_df, on='ISBN', how='inner')
joined_df = pd.merge(joined_df, users_df, on='User-ID', how='inner')

joined_df.shape, ratings_df.shape, books_df.shape, users_df.shape

In [ ]:
mn.matrix(users_df)

In [ ]:
error_rows = []

for index, value in enumerate(books_df['Year-Of-Publication']):
    try:
        books_df.at[index, 'Year-Of-Publication'] = int(value)
    except (ValueError, TypeError):
        error_rows.append(index)
wrong = books_df.iloc[error_rows]
wrong

In [ ]:
for idx in error_rows:
    book_title, author = books_df.iloc[idx]["Book-Title"].split(";")
    publisher = books_df.iloc[idx]["Year-Of-Publication"]
    year = books_df.iloc[idx]["Book-Author"]

    books_df.iloc[idx]["Book-Title"] = book_title
    books_df.iloc[idx]["Book-Author"] = author
    books_df.iloc[idx]["Year-Of-Publication"] = year

In [ ]:
books_df.iloc[error_rows]

## Counts and dist

In [ ]:
ratings_df["ISBN"].nunique(), books_df["ISBN"].nunique()

In [ ]:
rated_books = pd.merge(books_df, ratings_df, on="ISBN")
rated_books.shape, rated_books["ISBN"].nunique()

In [ ]:
# there are ratings for books which are not in book table
ratings_set = set(ratings_df["ISBN"])
books_set = set(books_df["ISBN"])

venn2(subsets=(len(ratings_set - books_set), len(books_set - ratings_set), len(ratings_set & books_set)),
      set_labels=("Ratings", "Books"),
      set_colors=("red", "blue"))

plt.show()

In [ ]:
num_isbns = 70405+270151

In [ ]:
# drop books without ratings
merged_books_df = books_df.merge(ratings_df, on='ISBN', how='left')

books_without_rating = merged_books_df[merged_books_df['Book-Rating'].isna()]

books_df = books_df[~books_df['ISBN'].isin(books_without_rating['ISBN'])]

In [ ]:
ratings_set = set(ratings_df["User-ID"])
users_set = set(users_df["User-ID"])

venn2(subsets=(len(ratings_set - users_set), len(users_set - ratings_set), len(ratings_set & users_set)),
      set_labels=("Ratings", "Users"),
      set_colors=("red", "blue"))

plt.show()

In [ ]:
# drop users without ratings
merged_df = users_df.merge(right=ratings_df, on='User-ID', how='left')

users_without_rating = merged_df[merged_df['Book-Rating'].isna()]

print("Users without a rating:")
print(users_without_rating[['User-ID']].shape)

In [ ]:
users_without_rating.head()

In [ ]:
users_df = users_df[~users_df['User-ID'].isin(users_without_rating['User-ID'])]

In [ ]:
res = ratings_df[ratings_df['User-ID'] == 1]
res

In [ ]:
plt.hist(ratings_df['Book-Rating'], bins=10, edgecolor='black', alpha=0.7)

plt.title('Distribution of Ratings')
plt.xlabel('Rating')
plt.ylabel('Frequency')

plt.show()

- rating 0 means implicit feedback. What does it mean?
- excerpt from the original paper
  - "Ratings can either be explicit, i.e., by
having the user state his opinion about a given product, or
implicit, when the mere act of purchasing or mentioning of
an item counts as an expression of appreciation. While implicit ratings are generally more facile to collect, their usage
implies adding noise to the collected information "
  - Data Collection
    - In a 4-week crawl, we collected data on 278, 858 members
of BookCrossing and 1, 157, 112 ratings, both implicit and
explicit, referring to 271, 379 distinct ISBNs 
- ok :)

In [ ]:
books_df.columns, books_df.shape

In [ ]:
books_df["Book-Title"].value_counts()

In [ ]:
books_df["Book-Author"].value_counts()

In [ ]:
books_df["Year-Of-Publication"].value_counts(), books_df["Year-Of-Publication"].astype(int).sort_values().unique()

In [ ]:
users_df.columns, users_df.shape

In [ ]:
# there were some really old people xd
users_df["Age"].value_counts(), users_df["Age"].dropna().sort_values().unique()

In [ ]:
ratings_df['User-ID'].value_counts()

## Aggregate into one row
- in one row information about user and book for simpler processing of models

In [ ]:
ratings_df.shape

In [ ]:
ratings_df["ISBN"].nunique()

In [ ]:
ratings = ratings_df.merge(books_df, how="left", on="ISBN")
ratings.shape

In [ ]:
ratings = ratings.merge(users_df, how="inner", on="User-ID")
ratings.shape

In [ ]:
ratings[ratings["Book-Title"] == "Why Didn't They Ask Evans?"]

In [ ]:
user_id_mapping = {user_id: index for index, user_id in enumerate(ratings['User-ID'].unique())}
ratings.loc[:, 'user_id'] = ratings['User-ID'].map(user_id_mapping)

book_id_mapping = {book_id: index for index, book_id in enumerate(ratings['ISBN'].unique())}
ratings.loc[:, 'book_id'] = ratings['ISBN'].map(book_id_mapping)

In [ ]:
ratings.describe()

In [ ]:
ratings_df["ISBN"].nunique(), ratings_df.shape

In [ ]:
users_df["User-ID"].nunique(), users_df.shape

In [ ]:
ratings["book_id"].nunique(), ratings["user_id"].nunique(), ratings["ISBN"].nunique()

In [ ]:
ratings.shape

In [ ]:
ratings.drop(["User-ID"], axis=1, inplace=True)
ratings.drop(["ISBN"], axis=1, inplace=True)

In [ ]:
ratings

In [ ]:
columns_to_check = ['Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher']
ratings = ratings.dropna(subset=columns_to_check)
ratings.isna().sum()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.hist(ratings['user_id'].value_counts(), bins=50, color='skyblue', edgecolor='black')
plt.title('Distribution of Number of Ratings per User')
plt.xlabel('Number of Ratings')
plt.ylabel('Number of Users')
plt.yscale('log')
plt.grid(True)
plt.show()

- drop users with less than 10 ratings

In [ ]:
user_ratings_counts = ratings['user_id'].value_counts()

users_with_less_than_10_ratings = user_ratings_counts[user_ratings_counts < 10].index
ratings = ratings[~ratings['user_id'].isin(users_with_less_than_10_ratings)]


ratings.shape, ratings["user_id"].nunique()

In [ ]:
book_ratings_counts = ratings['book_id'].value_counts()

books_with_less_than_10_ratings = book_ratings_counts[book_ratings_counts < 10].index
ratings = ratings[~ratings['book_id'].isin(books_with_less_than_10_ratings)]

print(ratings.shape)
ratings["book_id"].nunique(), ratings["user_id"].nunique()

In [ ]:
user_ratings_counts = ratings['user_id'].value_counts()

users_with_less_than_10_ratings = user_ratings_counts[user_ratings_counts <= 3].index
ratings = ratings[~ratings['user_id'].isin(users_with_less_than_10_ratings)]


ratings.shape, ratings["book_id"].nunique(), ratings["user_id"].nunique()

In [ ]:
book_ratings_counts

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.hist(ratings['user_id'].value_counts(), bins=50, color='skyblue', edgecolor='black')
plt.title('Distribution of Number of Ratings per User')
plt.xlabel('Number of Ratings')
plt.ylabel('Number of Users')
plt.yscale('log')
plt.grid(True)
plt.show()

In [ ]:
ratings['user_id'].value_counts()

In [ ]:
plt.hist(ratings[ratings["user_id"] == 5063]['Book-Rating'], bins=10, edgecolor='black', alpha=0.7)

In [ ]:
ratings[ratings["user_id"] == 5063].head(1)

In [ ]:
plt.hist(ratings[ratings["user_id"] == 75665]['Book-Rating'], bins=10, edgecolor='black', alpha=0.7)

In [ ]:
ratings[ratings["user_id"] == 75665].head(1)

In [ ]:
ratings["Book-Rating"].value_counts()

In [ ]:
ratings["Age"] = ratings["Age"].fillna(-1).astype(int)

In [ ]:
ratings["Age"].value_counts().sort_index()

In [ ]:
plt.hist(ratings[ratings["Age"] != -1]["Age"], bins="auto", edgecolor='black', alpha=0.7)
plt.show()

In [ ]:
ratings[ratings["Age"] > 123].shape[0]

In [ ]:
ratings.isna().sum()

In [ ]:
# for n in ["Book-Title", "Book-Author", "Publisher"]:
#     ratings[n] = ratings[n].fillna("")

In [ ]:
ratings["Year-Of-Publication"] = ratings["Year-Of-Publication"].fillna(-1)

In [ ]:
ratings.isna().sum()

In [ ]:
user_id_mapping = {user_id: index for index, user_id in enumerate(ratings['user_id'].unique())}
ratings.loc[:, 'user_id'] = ratings['user_id'].map(user_id_mapping)

book_id_mapping = {book_id: index for index, book_id in enumerate(ratings['book_id'].unique())}
ratings.loc[:, 'book_id'] = ratings['book_id'].map(book_id_mapping)

## Save

In [ ]:
ratings

In [ ]:
ratings.columns

In [ ]:
duplicate_pairs = ratings.duplicated(subset=['book_id', 'user_id'], keep=False)
# duplicate_pairs = ratings.duplicated(subset=['User-ID', 'ISBN'], keep=False)

print("Rows with duplicate book_id and user_id pairs:")
print(ratings[duplicate_pairs])

In [ ]:
ratings.to_csv(path_or_buf=new_ds_path)

## Notes on final ds
- missing Age -> -1
- missing Year of publication -> -1
- not all ratings have Book data